# Data Wrangling- Trips

### Import Data

In [1]:
%matplotlib inline

import matplotlib
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import datetime
import math

import seaborn as sns
sns.set()

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
print('Loading Trip Data...')

try:
    file_path_slug = '../../../datasets/bayareabikeshare/*_trip_data.csv'

    # glob all files
    file_list = glob(file_path_slug)

    trip_import = pd.DataFrame()

    counter = 1
    chunks = []

    # load data from each file
    for file in file_list:

        # import file in chunks
        for chunk in pd.read_csv(file, chunksize=10000, iterator=True):

            # set chunk index column to 'Trip ID'
            chunk = chunk.set_index('Trip ID')

            # define Columns
            chunk.columns = ['Duration', 'Start Date', 'Start Station', 'Start Terminal', 'End Date', 
                             'End Station', 'End Terminal', 'Bike #', 'Subscriber Type', 'Zip Code']

            # append chunk to chunks list
            chunks.append(chunk)

        print('\tFinished file! (%d of %d)' % (counter, len(file_list)))
        counter += 1

    # concat chunks
    trip_import = pd.concat(chunks)

    print('Data Loaded Successfully!')

except:
    print('oops... something went wrong importing the data :(')

Loading Trip Data...
	Finished file! (1 of 4)
	Finished file! (2 of 4)
	Finished file! (3 of 4)
	Finished file! (4 of 4)
Data Loaded Successfully!


In [4]:
trip_import.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 983648 entries, 913465 to 198776
Data columns (total 10 columns):
Duration           983648 non-null int64
Start Date         983648 non-null object
Start Station      983648 non-null object
Start Terminal     983648 non-null int64
End Date           983648 non-null object
End Station        983648 non-null object
End Terminal       983648 non-null int64
Bike #             983648 non-null int64
Subscriber Type    983648 non-null object
Zip Code           976838 non-null object
dtypes: int64(4), object(6)
memory usage: 82.6+ MB


### Clean Data

In [5]:
# zipcodes are all over the place, only keep corrected 5 digit zipcodes, and replace all others with NaNs
def clean_zipcode(item):
    if len(item) != 5:

        # split on '-'
        try:
            result = item.split('-')[0]
        except:
            result = item

        # split on '.'
        try:
            result = item.split('.')[0]
        except:
            result = item
        
        # if len of item is less than 5, return 'NaN'
        if len(result) < 5:
            result = 'NaN'
        else:
            # if len result is greater than 5, take at most, first 5 digits
            result = result[:5]
    else:
        result = item
    
    # make sure result is all digits
    if result.isdigit():
        return result
    else:
        return 'NaN'

In [6]:
print('Trip Data Cleanup Started...')
trip_data = trip_import.copy()

# cleanup column names
print('\tcleaning column names')
new_cols = []
for col in trip_data.columns:
    new_cols.append(col.replace(' ', '_').lower())
trip_data.columns = new_cols

# extract columns we want to keep
print('\tsubsetting to useful columns')
important_cols = ['duration', 'start_date', 'start_terminal', 'start_station', 'end_date', 'end_terminal', 'end_station', 'bike_#', 'subscriber_type', 'zip_code']
trip_data = trip_data[important_cols]

# we are only looking at stations in San Francisco
# sf_trips_data = trip_data[trip_data['start_terminal'].isin(sf_stations)]
# sf_trips_data = sf_trips_data[sf_trips_data['end_terminal'].isin(sf_stations)]

# trip_data = sf_trips_data.copy()

# create duration minutes column
print('\tcreating a duration_minutes column')
trip_data['duration_minutes'] = trip_data['duration'] / 60.0

# convert end and start dates to datetime objects
print('\tconverting end and start dates to datetime objects')
trip_data['start_date'] = pd.to_datetime(trip_data['start_date'], format="%m/%d/%Y %H:%M")
trip_data['end_date']   = pd.to_datetime(trip_data['end_date'],   format="%m/%d/%Y %H:%M")


# convert and clean zipcodes
print('\tcleaning zipcodes')
trip_data['zip_code'] = trip_data['zip_code'].astype(str)
trip_data.zip_code = trip_data.zip_code.apply(clean_zipcode)
trip_data['zip_code'] = pd.to_numeric(trip_data['zip_code'], errors='coerce')

# clean up data types
print('cleaning up data types')

trip_data['duration']         = trip_data['duration'].astype('float')
trip_data['start_terminal']   = trip_data['start_terminal'].astype('category')
trip_data['end_terminal']     = trip_data['end_terminal'].astype('category')
trip_data['bike_#']           = trip_data['bike_#'].astype('int')
trip_data['subscriber_type']  = trip_data['subscriber_type'].astype('category')
trip_data['zip_code']         = trip_data['zip_code'].astype('str')
trip_data['duration_minutes'] = trip_data['duration_minutes'].astype('float')

trip_data.rename(columns={'bike_#': 'bike_id'}, inplace=True)
trip_data.rename(columns={'zip_code': 'user_zip'}, inplace=True)
trip_data.rename(columns={'subscriber_type': 'user_type'}, inplace=True)

print('Trip Data Cleanup complete')

Trip Data Cleanup Started...
	cleaning column names
	subsetting to useful columns
	creating a duration_minutes column
	converting end and start dates to datetime objects
	cleaning zipcodes
cleaning up data types
Trip Data Cleanup complete


In [7]:
trip_data_sorted = trip_data.sort_values('start_date')

In [8]:
trip_data_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 983648 entries, 4069 to 1338408
Data columns (total 11 columns):
duration            983648 non-null float64
start_date          983648 non-null datetime64[ns]
start_terminal      983648 non-null category
start_station       983648 non-null object
end_date            983648 non-null datetime64[ns]
end_terminal        983648 non-null category
end_station         983648 non-null object
bike_id             983648 non-null int64
user_type           983648 non-null category
user_zip            983648 non-null object
duration_minutes    983648 non-null float64
dtypes: category(3), datetime64[ns](2), float64(2), int64(1), object(3)
memory usage: 70.4+ MB


In [9]:
trip_data_sorted.head(1)

,duration,start_date,start_terminal,start_station,end_date,end_terminal,end_station,bike_id,user_type,user_zip,duration_minutes
Trip ID,,,,,,,,,,,
4069,174.0,2013-08-29 09:08:00,64,2nd at South Park,2013-08-29 09:11:00,64,2nd at South Park,288,Subscriber,94114.0,2.9


In [10]:
trip_data_sorted.tail(1)

,duration,start_date,start_terminal,start_station,end_date,end_terminal,end_station,bike_id,user_type,user_zip,duration_minutes
Trip ID,,,,,,,,,,,
1338408,378.0,2016-08-31 23:32:00,46,Washington at Kearny,2016-08-31 23:38:00,60,Embarcadero at Sansome,667,Subscriber,94111.0,6.3


### Prune Trip Duration

In [ ]:
# prune data to exclude trips longer than 60 minutes
print('pruning data to trips no more than 60 minutes long...')
trip_data = trip_data_sorted[trip_data_sorted['duration_minutes'] <= 60].copy()

# Cleanup
trip_data.sort_index(inplace=True)
print('\tpruned data set \'trips\' consists of %i entries' % len(trip_data.index))

plt.subplots(figsize=(12,6))
ax = sns.distplot(trip_data[trip_data.user_type == 'Subscriber'].start_date.dt.hour, color='b', label='Subscribers')
sns.distplot(trip_data[trip_data.user_type == 'Customer'].start_date.dt.hour, color='r', label='Customers', ax=ax)
ax.set(xlabel='start hour')
plt.legend()
plt.show()

In [ ]:
955557/983648

In [ ]:
trip_data.to_csv('../../../datasets/bayareabikeshare/CLEANED/trip_data_cleaned.csv', encoding='utf-8')

In [ ]:
# Split out only Subscriber Trips
trip_data_subscribers = trip_data[trip_data.user_type == 'Subscriber']

# Split out only Customer Trips
trip_data_customers = trip_data[trip_data.user_type == 'Customer']

In [ ]:
trip_data_subscribers.to_csv('../../../datasets/bayareabikeshare/CLEANED/trip_data_cleaned_subscribers.csv', encoding='utf-8')
trip_data_customers.to_csv('../../../datasets/bayareabikeshare/CLEANED/trip_data_cleaned_customers.csv', encoding='utf-8')

In [ ]:
hourly_sub_trips = trip_data_subscribers.groupby(trip_data_subscribers['start_date'].dt.hour)['bike_id'].sum().to_frame()
hourly_sub_trips.plot(kind='bar', color='b', figsize=(15,6))
plt.show()

hourly_cust_trips = trip_data_customers.groupby(trip_data_customers['start_date'].dt.hour)['bike_id'].sum().to_frame()
hourly_cust_trips.plot(kind='bar', color='r', figsize=(15,6))
plt.show()

### Split up Subscriber Morning and Evening Commute Time Data

In [ ]:
sub_morning_commuters = trip_data_subscribers[trip_data_subscribers.start_date.dt.hour >= 7].copy()
sub_morning_commuters = sub_morning_commuters[sub_morning_commuters.end_date.dt.hour <= 11]
print(len(sub_morning_commuters))

sub_evening_commuters = trip_data_subscribers[trip_data_subscribers.start_date.dt.hour >= 16].copy()
sub_evening_commuters = sub_evening_commuters[sub_evening_commuters.end_date.dt.hour <= 20]
print(len(sub_evening_commuters))

# Add some additional information to Commuter trips

### Append start and end terminal zipcodes

In [ ]:
# import cleaned station data

station_data = pd.DataFrame()
station_data = pd.read_csv('../../../datasets/bayareabikeshare/CLEANED/station_data_cleaned.csv')

In [ ]:
station_data.head()

### Morning Commute Station Information

In [ ]:
sub_morning_commuters = trip_data_subscribers[trip_data_subscribers.start_date.dt.hour >= 7].copy()
sub_morning_commuters = sub_morning_commuters[sub_morning_commuters.end_date.dt.hour <= 11]
sub_morning_commuters.reset_index(inplace=True)

sub_morning_commuters.head()


In [ ]:
station_subset = station_data[['station_id', 'name', 'lat', 'long', 'dockcount', 'landmark', 'zip_code', 'lat_long']]
station_subset.head(40)



In [ ]:
# add start_terminal info
sub_morning_commuters = pd.merge(sub_morning_commuters, 
                  station_subset, 
                  how='left', 
                  left_on=['start_terminal', 'start_station'], 
                  right_on=['station_id', 'name'])




In [ ]:
nans = lambda df: df[df.isnull().any(axis=1)]

In [ ]:
nans(sub_morning_commuters)

In [ ]:
# drop station_id and name
sub_morning_commuters.drop(['station_id', 'name'], axis=1, inplace=True)

# rename columns for start terminal
sub_morning_commuters.rename(columns={'lat' : 'start_terminal_lat', 'long' : 'start_terminal_long', 'dockcount' : 'start_terminal_dockcount', 'landmark' : 'start_terminal_landmark', 'zip_code' : 'start_terminal_zip_code', 'lat_long' : 'start_terminal_lat_long'}, inplace=True)

# add end_terminal info
sub_morning_commuters = pd.merge(sub_morning_commuters, 
                  station_subset, 
                  how='left', 
                  left_on=['end_terminal', 'end_station'], 
                  right_on=['station_id', 'name'])
# drop station_id and name
sub_morning_commuters.drop(['station_id', 'name'], axis=1, inplace=True)

# rename columns for start terminal
sub_morning_commuters.rename(columns={'lat' : 'end_terminal_lat', 'long' : 'end_terminal_long', 'dockcount' : 'end_terminal_dockcount', 'landmark' : 'end_terminal_landmark', 'zip_code' : 'end_terminal_zip_code', 'lat_long' : 'end_terminal_lat_long'}, inplace=True)

# set index
sub_morning_commuters.set_index('Trip ID', inplace=True)
sub_morning_commuters.info()

sub_morning_commuters.to_csv('../../../datasets/bayareabikeshare/CLEANED/trip_data_cleaned_morning_commuters.csv', encoding='utf-8')



In [ ]:
customer_daylight_trips = trip_data_customers[trip_data_customers.start_date.dt.hour >= 8].copy()
customer_daylight_trips = customer_daylight_trips[customer_daylight_trips.end_date.dt.hour <= 19]
customer_daylight_trips.reset_index(inplace=True)

customer_daylight_trips.info()

In [ ]:
sub_morning_commuters = trip_data_subscribers[trip_data_subscribers.start_date.dt.hour >= 7].copy()
sub_morning_commuters = sub_morning_commuters[sub_morning_commuters.end_date.dt.hour <= 11]
sub_morning_commuters.reset_index(inplace=True)

station_subset = station_data[['station_id', 'name', 'lat', 'long', 'dockcount', 'landmark', 'zip_code', 'lat_long']]

# add start_terminal info
sub_morning_commuters = pd.merge(sub_morning_commuters, 
                  station_subset, 
                  how='left', 
                  left_on=['start_terminal', 'start_station'], 
                  right_on=['station_id', 'name'])

# drop station_id and name
sub_morning_commuters.drop(['station_id', 'name'], axis=1, inplace=True)

# rename columns for start terminal
sub_morning_commuters.rename(columns={'lat' : 'start_terminal_lat', 'long' : 'start_terminal_long', 'dockcount' : 'start_terminal_dockcount', 'landmark' : 'start_terminal_landmark', 'zip_code' : 'start_terminal_zip_code', 'lat_long' : 'start_terminal_lat_long'}, inplace=True)

# add end_terminal info
sub_morning_commuters = pd.merge(sub_morning_commuters, 
                  station_subset, 
                  how='left', 
                  left_on=['end_terminal', 'end_station'], 
                  right_on=['station_id', 'name'])
# drop station_id and name
sub_morning_commuters.drop(['station_id', 'name'], axis=1, inplace=True)

# rename columns for start terminal
sub_morning_commuters.rename(columns={'lat' : 'end_terminal_lat', 'long' : 'end_terminal_long', 'dockcount' : 'end_terminal_dockcount', 'landmark' : 'end_terminal_landmark', 'zip_code' : 'end_terminal_zip_code', 'lat_long' : 'end_terminal_lat_long'}, inplace=True)

# set index
sub_morning_commuters.set_index('Trip ID', inplace=True)
sub_morning_commuters.info()

sub_morning_commuters.to_csv('../../../datasets/bayareabikeshare/CLEANED/trip_data_cleaned_morning_commuters.csv', encoding='utf-8')


### Evening Commute Station Information

In [ ]:
sub_evening_commuters = trip_data_subscribers[trip_data_subscribers.start_date.dt.hour >= 16].copy()
sub_evening_commuters = sub_evening_commuters[sub_evening_commuters.end_date.dt.hour <= 20]
sub_evening_commuters.reset_index(inplace=True)
station_subset = station_data[['station_id', 'name', 'lat', 'long', 'dockcount', 'landmark', 'zip_code', 'lat_long']]

# add start_terminal info
sub_evening_commuters = pd.merge(sub_evening_commuters, 
                  station_subset, 
                  how='left', 
                  left_on=['start_terminal', 'start_station'], 
                  right_on=['station_id', 'name'])

# drop station_id and name
sub_evening_commuters.drop(['station_id', 'name'], axis=1, inplace=True)

# rename columns for start terminal
sub_evening_commuters.rename(columns={'lat' : 'start_terminal_lat', 
                                      'long' : 'start_terminal_long', 
                                      'dockcount' : 'start_terminal_dockcount', 
                                      'landmark' : 'start_terminal_landmark', 
                                      'zip_code' : 'start_terminal_zip_code', 
                                      'lat_long' : 'start_terminal_lat_long'}, inplace=True)

# add end_terminal info
sub_evening_commuters = pd.merge(sub_evening_commuters, 
                  station_subset, 
                  how='left', 
                  left_on=['end_terminal', 'end_station'], 
                  right_on=['station_id', 'name'])
# drop station_id and name
sub_evening_commuters.drop(['station_id', 'name'], axis=1, inplace=True)

# rename columns for start terminal
sub_evening_commuters.rename(columns={'lat' : 'end_terminal_lat', 
                                      'long' : 'end_terminal_long', 
                                      'dockcount' : 'end_terminal_dockcount', 
                                      'landmark' : 'end_terminal_landmark', 
                                      'zip_code' : 'end_terminal_zip_code', 
                                      'lat_long' : 'end_terminal_lat_long'}, inplace=True)

# set index
sub_evening_commuters.set_index('Trip ID', inplace=True)
sub_evening_commuters.info()

sub_evening_commuters.to_csv('../../../datasets/bayareabikeshare/CLEANED/trip_data_cleaned_evening_commuters.csv', encoding='utf-8')

## Load Cleaned Commuter Trip Files

In [ ]:
# Load Cleaned Files

morning_commuter_trips = pd.read_csv('../../../datasets/bayareabikeshare/CLEANED/trip_data_cleaned_morning_commuters.csv', parse_dates=['start_date', 'end_date'])
evening_commuter_trips = pd.read_csv('../../../datasets/bayareabikeshare/CLEANED/trip_data_cleaned_evening_commuters.csv', parse_dates=['start_date', 'end_date'])

In [ ]:
morning_commuter_trips.head()

In [ ]:
junk = morning_commuter_trips[morning_commuter_trips['start_terminal_zip_code'].isnull()]
junk.head()

### Append Weather Data to trips

In [ ]:
# import cleaned weather data
weather_data = pd.DataFrame()
weather_data = pd.read_csv('../../../datasets/bayareabikeshare/CLEANED/weather_cleaned_all.csv', parse_dates=['date'])

In [ ]:
evening_commuter_trips_weather = pd.merge(evening_commuter_trips, 
                                          weather_data, 
                                          how='left', 
                                          left_on=[evening_commuter_trips['start_date'].dt.date, 'start_terminal_zip_code'], 
                                          right_on=[weather_data['date'].dt.date, 'zip'])
evening_commuter_trips_weather.drop(['date', 'zip'], axis=1, inplace=True)
evening_commuter_trips_weather.set_index('Trip ID', inplace=True)



morning_commuter_trips_weather = pd.merge(morning_commuter_trips, 
                                          weather_data, 
                                          how='left', 
                                          left_on=[morning_commuter_trips['start_date'].dt.date, 'start_terminal_zip_code'], 
                                          right_on=[weather_data['date'].dt.date, 'zip'])
morning_commuter_trips_weather.drop(['date', 'zip'], axis=1, inplace=True)
morning_commuter_trips_weather.set_index('Trip ID', inplace=True)



In [ ]:
evening_commuter_trips_weather.to_csv('../../../datasets/bayareabikeshare/CLEANED/trip_data_cleaned_evening_commuters_weather.csv', encoding='utf-8')
morning_commuter_trips_weather.to_csv('../../../datasets/bayareabikeshare/CLEANED/trip_data_cleaned_morning_commuters_weather.csv', encoding='utf-8')

In [ ]:
evening_commuter_trips_weather

## Append weather and station data to all trips

In [ ]:
trips = trip_data.copy()

print(len(trips))


trips.reset_index(inplace=True)
station_subset = station_data[['station_id', 'name', 'lat', 'long', 'dockcount', 'landmark', 'zip_code', 'lat_long']]


print('appending start terminal info')
# add start_terminal info
trips = pd.merge(trips, 
                  station_subset, 
                  how='left', 
                  left_on=['start_terminal', 'start_station'], 
                  right_on=['station_id', 'name'])



# drop station_id and name
trips.drop(['station_id', 'name'], axis=1, inplace=True)


# rename columns for start terminal
trips.rename(columns={'lat' : 'start_terminal_lat', 'long' : 'start_terminal_long', 'dockcount' : 'start_terminal_dockcount', 'landmark' : 'start_terminal_landmark', 'zip_code' : 'start_terminal_zip_code', 'lat_long' : 'start_terminal_lat_long'}, inplace=True)




print('appending end terminal info')
# add end_terminal info
trips = pd.merge(trips, 
                  station_subset, 
                  how='left', 
                  left_on=['end_terminal', 'end_station'], 
                  right_on=['station_id', 'name'])
# drop station_id and name
trips.drop(['station_id', 'name'], axis=1, inplace=True)

# rename columns for start terminal
trips.rename(columns={'lat' : 'end_terminal_lat', 'long' : 'end_terminal_long', 'dockcount' : 'end_terminal_dockcount', 'landmark' : 'end_terminal_landmark', 'zip_code' : 'end_terminal_zip_code', 'lat_long' : 'end_terminal_lat_long'}, inplace=True)



print('appending weather info')
trips = pd.merge(trips, 
                          weather_data, 
                          how='left', 
                          left_on=[trips['start_date'].dt.date, 'start_terminal_zip_code'], 
                          right_on=[weather_data['date'].dt.date, 'zip'])
trips.drop(['date', 'zip'], axis=1, inplace=True)
trips.set_index('Trip ID', inplace=True)


trips.info()

In [ ]:
trips.to_csv('../../../datasets/bayareabikeshare/CLEANED/trip_data_cleaned_master.csv', encoding='utf-8')

In [ ]:
trips.info()